In [184]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
import numpy as np
import matplotlib.pyplot as plt

##Load the dataset

In [185]:
#malaria dataset 
(train_data_raw, test_data_raw),info =tfds.load("malaria",split = ["train[:80%]","train[80%:]"], with_info=True)

In [186]:
train_data = train_data_raw 
test_data = test_data_raw 

In [187]:
def unzip(data):
  a = data.map(lambda x: x["image"])
  b = data.map(lambda x: x["label"])
  return a,b
  

In [188]:
def data_handling(data, batchsize = 64):
  #unzip data
  images, labels = unzip(data)

  #normalize images
  images = images.map(lambda img: tf.image.per_image_standardization(img))

  #resizing to equal size
  images = images.map(lambda img: tf.image.resize_with_crop_or_pad(img, 394,394))
  
  #onehotify the labels 
  labels = labels.map(lambda x: tf.reshape(tf.one_hot(x, 2), (-1,)))
  
  #zip images and labels for batching and shuffling 
  ds = tf.data.Dataset.zip((images,labels))
  ds.prefetch(4)

  #batch and shuffle data set
  ds = ds.batch(batch_size = batchsize)
  ds = ds.shuffle(buffer_size = batchsize)

  return ds

In [189]:
train_data = data_handling(train_data)
test_data = data_handling(test_data)

In [190]:
class Model(Model):
  '''Class for implementing a Neural Network with convolutional layers'''

  def __init__(self):
      super(Model, self).__init__()
      self.hid_conv_1 = tf.keras.layers.Conv2D(
          filters=20,kernel_size= 3, 
          input_shape = (64, 394, 394, 3), 
          activation= tf.keras.activations.relu)
      
      self.max_pool_1 = tf.keras.layers.MaxPooling2D()

      self.hid_conv_2 = tf.keras.layers.Conv2D(
          filters = 60, kernel_size = 3, 
          activation = tf.keras.activations.relu
      )

      self.max_pool_2 = tf.keras.layers.MaxPooling2D()

      self.hid_conv_3 = tf.keras.layers.Conv2D(
          filters = 100, kernel_size = 3, 
          activation = tf.keras.activations.relu
      )

      self.max_pool_3 = tf.keras.layers.MaxPooling2D()

      self.hid_conv_4 = tf.keras.layers.Conv2D(
          filters = 100, kernel_size = 3, 
          activation = tf.keras.activations.relu
      )

      self.max_pool_4 = tf.keras.layers.MaxPooling2D()

      self.global_pool = tf.keras.layers.GlobalAveragePooling2D()

      self.read_out_layer = tf.keras.layers.Dense(2, activation= tf.keras.activations.softmax)

  def call(self,x):     
      x = self.hid_conv_1(x)        
      x = self.max_pool_1(x)
      x = self.hid_conv_2(x)
      x = self.max_pool_2(x)
      x = self.hid_conv_3(x)
      x = self.max_pool_3(x)
      x = self.hid_conv_4(x)
      x = self.max_pool_4(x)
      x = self.global_pool(x)
      x = self.read_out_layer(x)
      return x


###Training the model



In [191]:
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss 

def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input)
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = np.mean(test_loss_aggregator)
  test_accuracy = np.mean(test_accuracy_aggregator)

  return test_loss, test_accuracy 
  

In [192]:
tf.keras.backend.clear_session()

### Hyperparameters
num_epochs = 25
learning_rate = 0.00001
running_average_factor = 0.95

# Initialize the model.
model = Model()
# Initialize the loss: binaray cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()
# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam(learning_rate)


# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_data, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_data, cross_entropy_loss)
train_losses.append(train_loss)

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print('Epoch: __ ' + str(epoch))

    train_data = train_data.shuffle(buffer_size=128)
    test_data = test_data.shuffle(buffer_size=128)

    #training (and checking in with training)
    running_average = 0
    for (input,target) in train_data:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        running_average = running_average_factor * running_average  + (1 - running_average_factor) * train_loss
    train_losses.append(running_average)

    #testing
    test_loss, test_accuracy = test(model, test_data, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

InvalidArgumentError: ignored

In [ ]:
# taken from Tensorflow_Intro.ipynb
# Visualize accuracy and loss for training and test data. 
# One plot training and test loss.
# One plot training and test accuracy.
plt.figure()
line1, = plt.plot(training_losses)
line2, = plt.plot(test_losses)
plt.xlabel("Training steps")
plt.ylabel("Loss")
plt.legend((line1,line2),("training","test"))
plt.show()

plt.figure()
line1, = plt.plot(test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Accuracy")
plt.show()